TitanQ API Key

In [1]:
# Enter your API Key Here
# Obtain your API key by contacting --> support@infinityq.tech
# Example: TITANQ_DEV_API_KEY = "00000000-0000-0000-0000-000000000000"
TITANQ_DEV_API_KEY = input("Please enter your API key (e.g. 00000000-0000-0000-0000-000000000000):")

User Specifications

In [ ]:
instance_name = "Enter Instance Name Here"
#Ex: "g000989"

num_runs = 1

instance_file_path = "Enter File Path Here" 
#Ex: "hss-overview-benchmarks/qubos"

Imports and Logging Setup

In [ ]:
# Prompt the user and wait for an answer before installing dependencies.
# Note: Usage of a virtual Python environment is recommended.
if input("Install Python dependencies? [y/n]").lower() == "y":
    print("Installing dependencies (this can take a few minutes)...")
    !pip install -r "requirements.txt" --upgrade
else:
    print("Skipping dependency installation.")

import numpy as np
import logging
from MQLib_utils import *
from titanq import Model, Vtype, Target

logging.getLogger('botocore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

Load Instance + Hyperparameters

In [ ]:
weight_matrix, bias_vector = generate_weights_bias(instance_name, instance_file_path)
T_min, T_max, coupling_mult, num_chains, num_engines = load_hyperparameters(instance_name)
betas = 1/(np.linspace(T_min, T_max, num_chains, dtype=np.float32))

TitanQ SDK Setup

In [ ]:
model = Model(api_key=TITANQ_DEV_API_KEY)
model.add_variable_vector('x', weight_matrix.shape[0], Vtype.BINARY)
model.set_objective_matrices(weight_matrix, bias_vector, Target.MINIMIZE)

Solver + Results

In [ ]:
print("--- RESULTS ---")
print("Instance:", instance_name)
print()

qubo_float64 = read_instance(instance_file_path + "/" + instance_name)

for i in range(num_runs):
    response = model.optimize(beta = betas, coupling_mult = coupling_mult, timeout_in_secs = 300, num_chains = num_chains, num_engines = num_engines)
    print("Run #:", i+1)
    
    min_idx = np.argmin(np.array(response.ising_energy))
    sol = np.array(response.x[min_idx], dtype=np.float64)
    calc_energy = np.dot(np.dot(sol, qubo_float64), sol)
    print("Minimum Ising Energy:", calc_energy)
    
    print("Solve Time(s):", response.computation_metrics()['solve_time_in_secs'])
    print()